# Model comparison

In this notebook we compare PEC corrections calculated with different ol-melt Fe-Mg Kd models

import the relevant packages

In [1]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

# this is needed to clean up printed results during looped calculations
from IPython.display import clear_output

load data and set up FeOi according to the [FeOi](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html) and [PEC correction](https://magmapec.readthedocs.io/en/latest/notebooks/pec_corr.html) examples.

In [2]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"
pressure_file ="./data/pressure.csv"
wholerock_file = "./data/wholerock.csv"

melt = mp.read_melt(melt_file, index_col=["name"], units="wt. %")
olivine = mp.read_olivine(olivine_file, index_col=["name"], units="wt. %")
pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()
wholerock = mp.read_melt(wholerock_file, index_col=["name"])

In [3]:
x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)
FeO_model = FeOi_predict.model

Here are all the Kd models we're going to test. You can also do this for Fe3Fe2 or melt thermometers instead with `mpc.Fe3Fe2_models` or `mpc.melt_thermometers`. 

In [4]:
Kd_models = mpc.Kd_ol_FeMg_models
Kd_models

['blundy2020',
 'fixed',
 'putirka2016_8a',
 'putirka2016_8b',
 'putirka2016_8c',
 'putirka2016_8d',
 'saper2022',
 'sun2020',
 'toplis2005']

Now we calculate PEC corrections iteratively for each Kd model. 

For a fixed Kd we also need to define the value and its error and pass them as a list to the configuration

Crystallization extents are stored in the `pec_results` dataframe and corrected melt compositions in the `corrected_inclusions` dictionary. This dictionary uses Kd model names as keys and stores compositions in dataframes.

In [5]:
pec_results = pd.DataFrame(index=melt.index, dtype=float)
corrected_inclusions = {}

Kd_value = 0.35
Kd_error = 0.02

for i, model in enumerate(Kd_models):

    # add Kd value and error if the model is 'fixed'
    set_model = model if model != "fixed" else (model, Kd_value, Kd_error)
    # set the Kd model
    mpc.model_configuration.Kd_model = set_model

    clear_output()
    print(f"model: {model}\n{i+1:03}/{len(Kd_models):03}")

    pec_model = mpc.PEC(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeO_model)
    melts_corrected, pec, checks = pec_model.correct()

    pec_results[set_model] = pec["total_crystallisation"]
    corrected_inclusions[set_model] = melts_corrected.copy()
    

model: toplis2005
009/009
Equilibrating ... |██████████████████████████████| 100% [10/10] in 6.2s 
Correcting    ... |██████████████████████████████| 100% [10/10] in 23.9s 


These are the crystallization extents calculated with the different models

In [6]:
pec_results

,blundy2020,"(fixed, 0.35, 0.02)",putirka2016_8a,putirka2016_8b,putirka2016_8c,putirka2016_8d,saper2022,sun2020,toplis2005
name,,,,,,,,,
PI032-04-01,9.683716,12.240234,10.934509,10.931458,10.926880,10.859961,6.580603,11.060718,9.810583
PI032-04-02,10.817993,13.950098,12.149658,12.149658,12.268896,12.062793,8.185342,12.224097,10.817993
PI041-02-02,1.162512,2.839575,1.933740,1.757172,1.930688,1.760223,-1.203491,1.762512,-0.004739
PI041-03-01,14.810913,17.620947,16.087646,15.307373,15.492114,14.013965,11.345508,17.995654,12.648291
PI041-03-03,14.344751,17.224976,15.768188,14.467151,15.256470,13.353906,11.000000,17.656470,12.132538
PI041-05-04,-2.550830,-1.160742,-1.796387,-2.499170,-2.147778,-2.491541,-4.983539,-2.153882,-4.402173
PI041-05-06,3.494727,5.709229,4.452222,3.711511,4.094727,3.349438,0.811664,4.502356,1.994775
PI041-07-01,13.548462,16.336914,14.454077,13.733203,14.263232,12.957617,9.792810,15.403174,12.423010
PI041-07-02,12.966553,15.680347,14.078271,13.157397,13.687427,12.427368,9.194116,15.361975,11.121533


And here are two examples of the associated corrected melt compositions

In [7]:
corrected_inclusions["blundy2020"]

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,,
PI032-04-01,48.943266,14.012585,7.633723,9.705223,10.314180,3.601978,0.672491,0.138303,2.456252,0.279324,0.000000,0.605733,1.388033,0.076000,0.140183,0.032727,100.0
PI032-04-02,48.479028,14.628248,7.551064,9.385928,10.340269,3.449335,0.909959,0.140869,2.566939,0.325472,0.000000,0.650645,1.287911,0.081172,0.160056,0.043103,100.0
PI041-02-02,49.035936,16.830519,5.175333,9.123028,10.256435,3.761809,1.070165,0.157858,2.784500,0.557061,0.000000,0.460638,0.650855,0.047316,0.067539,0.021008,100.0
PI041-03-01,45.804865,15.409914,7.867232,10.765410,10.735038,3.273248,1.119803,0.117423,3.084308,0.539915,0.000000,0.780261,0.286923,0.077759,0.084962,0.052939,100.0
PI041-03-03,45.225647,15.624239,7.726354,10.983136,10.806707,3.355998,1.146523,0.094427,3.203706,0.508564,0.000000,0.806328,0.305528,0.080241,0.078063,0.054539,100.0
PI041-05-04,47.738792,18.498460,4.169708,9.355941,9.388333,4.577086,1.598891,0.142974,2.474651,0.821224,0.000000,0.507599,0.459418,0.087334,0.120853,0.058735,100.0
PI041-05-06,46.350282,16.864223,5.160065,8.871397,11.454392,3.976527,1.407875,0.173510,3.611065,0.615574,0.000000,0.636027,0.582041,0.106738,0.127309,0.062975,100.0
PI041-07-01,45.737294,15.075423,7.646520,9.588052,11.634791,3.098222,1.259201,0.150570,3.495913,0.549115,0.000000,0.462876,1.010233,0.074322,0.161728,0.055741,100.0
PI041-07-02,45.805997,15.374675,7.403629,9.952190,11.491078,3.148886,1.342814,0.139517,3.489348,0.608197,0.000000,0.367433,0.608364,0.072635,0.138603,0.056633,100.0


In [8]:
corrected_inclusions["saper2022"]

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,,
PI032-04-01,49.203959,14.365494,6.644781,9.940741,10.226162,3.692745,0.689437,0.132979,2.518148,0.286362,0.000000,0.620997,1.423010,0.077916,0.143715,0.033552,100.0
PI032-04-02,48.642946,14.931485,6.648134,9.571975,10.398662,3.520777,0.928806,0.135970,2.620105,0.332213,0.000000,0.664121,1.314586,0.082853,0.163371,0.043995,100.0
PI041-02-02,49.253759,17.144352,4.387411,9.285701,10.181687,3.832088,1.090158,0.150197,2.836520,0.567468,0.000000,0.469243,0.663015,0.048200,0.068800,0.021400,100.0
PI041-03-01,45.991537,15.828690,6.793620,11.047368,10.673911,3.362497,1.150335,0.111175,3.168406,0.554636,0.000000,0.801536,0.294747,0.079879,0.087279,0.054382,100.0
PI041-03-03,45.389128,16.031548,6.641397,11.258756,10.802356,3.443830,1.176529,0.087661,3.287552,0.521874,0.000000,0.827430,0.313524,0.082341,0.080106,0.055967,100.0
PI041-05-04,47.880830,18.844346,3.377299,9.522770,9.334323,4.662877,1.628860,0.133975,2.521035,0.836617,0.000000,0.517113,0.468029,0.088971,0.123118,0.059836,100.0
PI041-05-06,46.479616,17.211279,4.261656,9.046157,11.480183,4.058779,1.436996,0.164840,3.685758,0.628307,0.000000,0.649183,0.594080,0.108946,0.129943,0.064278,100.0
PI041-07-01,45.869136,15.532809,6.520845,9.866430,11.590983,3.192415,1.297483,0.143338,3.602196,0.565809,0.000000,0.476948,1.040946,0.076581,0.166645,0.057436,100.0
PI041-07-02,45.928086,15.827018,6.311079,10.232945,11.445067,3.241655,1.382375,0.133205,3.592147,0.626115,0.000000,0.378258,0.626287,0.074775,0.142686,0.058301,100.0
